<img src="https://www.docker.com/wp-content/uploads/2022/03/horizontal-logo-monochromatic-white.png" width="400" height="200" align="left"> 

## Check images

In [5]:
%%bash
docker images

REPOSITORY          TAG                   IMAGE ID       CREATED         SIZE
micapipe            v0.1.5                88bc128c4d9e   2 days ago      36.7GB
my_fastsurfer       cpu                   6167e77f6578   6 weeks ago     5.49GB
deepmi/fastsurfer   surfonly-cpu-v2.0.0   a1373a0fbbe1   7 weeks ago     1.59GB
deepmi/fastsurfer   cpu-v2.0.0            28f6bd92ba08   7 weeks ago     5.49GB
micalab/micapipe    v0.1.4                1a581843a33a   3 months ago    36.7GB
ubuntu              20.04                 680e5dfb52c7   4 months ago    72.8MB
ubuntu              16.04                 b6f507652425   18 months ago   135MB
ubuntu              latest                f643c72bc252   2 years ago     72.9MB
ubuntu              bionic-20201119       2c047404e52d   2 years ago     63.3MB


## Remove images
```bash
docker rmi -f <docker-image>
```

## Get inside the container
```bash
docker run -it --entrypoint /bin/bash micapipe:v0.1.5
```

## Build container
From the main repository where the `Dockerfile` is locate, particularry in the case of `micapipe`, I run, `-t` is for tag:
```bash
docker build . -t micapipe:v0.1.5
```

## Taggin the version via docker
If the built was successful the next step is to tag the containe via docker:
```bash
docker tag micapipe:v0.1.5 micalab/micapipe:v0.1.5
```

## Deploy container to `dockerhub`
The final step is to release the new contianer to the docker hub, this step will require the user credentials e.g. `micalab`.
```bash
docker login --username micalab

docker push micalab/micapipe:v0.1.5
```

# Sigularity container <img src="https://sylabs.io/wp-content/uploads/2022/03/singularity-logo-round.svg" width="100" height="100" align="left">

### 

# Github actions 
## Create a container after each push on main (master)

# CirlceCI Continuous integration workflow
<img src="https://global-uploads.webflow.com/622659e406588ec12d1336d5/624789be8ca70284b471a6bd_circleci-ar21.png" width="100" height="100" align="left">